In [1]:
import xgboost as xgb
import utils as ut
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train_v1.csv')
test = pd.read_csv('test_v1.csv')

In [3]:
y = np.log(train['trip_duration'].values + 1)
train = train.drop(['Unnamed: 0', 'trip_duration'], axis=1)
test = test.drop(['Unnamed: 0'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
Xtr, Xv, ytr, yv = train_test_split(train, y, test_size=0.2, random_state=1992)

In [7]:
dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xv, label=yv)
dtest = xgb.DMatrix(test)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

In [8]:
xgb_pars = {'min_child_weight': 10,
            'eta': 0.005,
            'colsample_bytree': 0.3,
            'max_depth': 10,
            'subsample': 0.8,
            'lambda': 2.0,
            'nthread': -1,
            'booster' : 'gbtree',
            'silent': 1,
            'eval_metric': 'rmse',
            'objective': 'reg:linear'}

In [10]:
FOREVER_COMPUTING_FLAG = True
xgb_pars = []
for MCW in [10, 20]:
    for ETA in [0.05]:
        for CS in [0.3, 0.4, 0.5]:
            for MD in [6, 8, 10, 12, 15]:
                for SS in [0.9]:
                    for LAMBDA in [2.0]:
                        xgb_pars.append({'min_child_weight': MCW,
                                         'eta': ETA, 
                                         'colsample_bytree': CS,
                                         'max_depth': MD,
                                         'subsample': SS,
                                         'lambda': LAMBDA, 
                                         'nthread': -1,
                                         'booster' : 'gbtree',
                                         'eval_metric': 'rmse',
                                         'silent': 1,
                                         'objective': 'reg:linear'})

for xgb_par in xgb_pars:
    print(xgb_par)
    model = xgb.train(xgb_par, dtrain, 2000, watchlist, early_stopping_rounds=50,
                      maximize=False, verbose_eval=100)
    print('Modeling RMSLE %.5f' % model.best_score)

{'min_child_weight': 10, 'eta': 0.05, 'colsample_bytree': 0.3, 'max_depth': 6, 'subsample': 0.9, 'lambda': 2.0, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
[0]	train-rmse:5.72053	valid-rmse:5.72099
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[100]	train-rmse:0.393139	valid-rmse:0.39511
[200]	train-rmse:0.380825	valid-rmse:0.385329
[300]	train-rmse:0.375172	valid-rmse:0.381874
[400]	train-rmse:0.371214	valid-rmse:0.379984
[500]	train-rmse:0.367979	valid-rmse:0.378515
[600]	train-rmse:0.36528	valid-rmse:0.377527
[700]	train-rmse:0.362766	valid-rmse:0.376675
[800]	train-rmse:0.360372	valid-rmse:0.376011
[900]	train-rmse:0.358199	valid-rmse:0.375448
[1000]	train-rmse:0.356273	valid-rmse:0.374937
[1100]	train-rmse:0.354416	valid-rmse:0.374538
[1200]	train-rmse:0.352619	valid-rmse:0.374104
[1300]	train-rmse:0.350923	valid-rmse:0.3737
[140

KeyboardInterrupt: 